In [2]:
import numpy as np; import pandas as pd
import re; import datetime; from collections import Counter, OrderedDict
import category_encoders as ce
from sklearn.model_selection import train_test_split
import pylogit as pl

In [17]:
df = pd.read_csv(r'가마우지\race.csv', engine='python', encoding='utf-8')

In [75]:
df.columns

Index(['rank', 'lane', 'sex', 'age', 'jockey_w', 'dandivi', 'yeondivi',
       'cure_in_1m', 'weight_diff', 'raw_weight', 'weight_added', 'distance',
       'velocity', 'horse_level_num', 'groupid', 'adj_vel', 'prev_rank',
       'norating'],
      dtype='object')

In [19]:
df['groupid'] = df['date'] + df['round'].apply(str)

In [20]:
idset = list(set(df['groupid']))
give_ids = dict(list(zip(idset, list(range(len(idset))))))

In [21]:
df['groupid'] = df['groupid'].apply(lambda x: give_ids[x])
df = df[df['distance'] > 0] # 거리 정보 있는 것만 살림

In [34]:
distances = list(set(df['distance']))
avg_vel = {}=

In [42]:
for i in distances:
    avg_vel[i] = df[df['distance'] == i]['velocity'].mean()

In [48]:
df['avg_vel'] = df['distance'].apply(lambda x: avg_vel[x])
df['adj_vel'] = df['velocity'] / df['avg_vel']

In [67]:
nums = list(map(str, range(1,17)))
df = df[(df['prev1_rank'].apply(lambda x: x in nums)) & (df['prev2_rank'].apply(lambda x: x in nums))]

In [70]:
df['prev_rank'] = df['prev1_rank'] + df['prev2_rank']
del df['prev1_rank'], df['prev2_rank']

In [74]:
df['norating'] = (df['horse_level_num'] == 10).apply(int)

In [86]:
df['choice'] = (df['rank'] == 1).apply(int)

In [90]:
groupids = list(set(df['groupid']))
train, test = train_test_split(groupids, test_size = 0.3, random_state=datetime.datetime.now().second)

train = df[df['groupid'].apply(lambda x: x in train)]
test = df[df['groupid'].apply(lambda x: x in test)]

In [94]:
train2 = train[train['rank']>1]
train2['choice'] = (train2['rank'] == 2).apply(int)
train2['groupid'] = train2['groupid'] + 10000

train3 = train[train['rank']>2]
train3['choice'] = (train3['rank'] == 3).apply(int)
train3['groupid'] = train3['groupid'] + 20000

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [96]:
train = train.append(train2, ignore_index=True).append(train3, ignore_index=True)

In [121]:
train.columns

Index(['lane', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m',
       'weight_diff', 'raw_weight', 'weight_added', 'horse_level_num',
       'groupid', 'prev_rank', 'norating', 'choice', 'intercept', 'sex1',
       'sex2'],
      dtype='object')

In [99]:
del train['rank'], train['distance'], train['velocity']

In [200]:
basic_specification = OrderedDict()
basic_names = OrderedDict()
li = list(range(1,17))
xs = ['lane', 'sex1', 'sex2', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'raw_weight', 'weight_added', 'horse_level_num', 'prev_rank', 'norating']
basic_specification["intercept"] = [li]
basic_names['intercept'] = ['annoying']

for i in xs:
    basic_specification[i] = [li]
    basic_names[i] = [i]

In [135]:
custom_alt_id = "lane"
obs_id_column = "groupid"
choice_column = "choice"

In [202]:
clogit_mnl = pl.create_choice_model(data=train_new.sort_values(by=['groupid']),
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

clogit_mnl.fit_mle(np.ones(14))
clogit_mnl.get_statsmodels_summary()

Log-likelihood at zero: -30,146.3572
Initial Log-likelihood: -1,138,601.4770


C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:420: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 3.30 seconds.
Final log-likelihood: -32,563.6512


MemoryError: 

In [6]:
test['sex1'] = (test['sex'] == '수').apply(int)
test['sex2'] = (test['sex'] == '암').apply(int)
del test['sex']

In [119]:
train = train[train['horse_level_num'] > 0]

In [5]:
def rank(x):
    y=str(x)
    if len(y) == 2:
        return int(y[0])+int(y[1])
    if len(y) == 3:
        return int(y[:2])+int(y[2])
    else:
        return int(y[:2])+int(y[2:])
test['prev_rank'] = test['prev_rank'].apply(rank)

In [175]:
groupids = list(set(train['groupid']))

In [189]:
import random
tmps = []; memory = []

for i in groupids:
    dta = train[train['groupid']==i]
    diff = list(dta['lane'])
    dta = dta[dta['choice'] == 0]
    if len(dta)==0: 
        memory.append(i)
        continue
    for j in li:
        if j not in diff:
            r = random.randint(0, len(dta)-1)
            tmp=list(dta.iloc[r,:])
            tmp[0]=j
            tmps.append(tmp)

In [194]:
train_new = train.append(pd.DataFrame(tmps, columns = train.columns), ignore_index = True)

In [198]:
train_new = train_new[train_new['식별'] == False]

In [196]:
train_new['식별'] = train_new['groupid'].apply(lambda x: x in memory)

In [199]:
train_new

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,intercept,sex1,sex2,식별
0,10,4.0,54.0,1.9,1.2,0.0,447.0,-7.0,3.0,1136.0,8.0,0.0,1.0,1.0,1.0,0.0,False
1,3,6.0,54.5,40.2,8.1,0.0,480.0,-1.0,3.0,1136.0,7.0,0.0,0.0,1.0,0.0,0.0,False
2,11,3.0,55.5,4.7,2.0,1.0,463.0,-5.0,3.0,1136.0,6.0,0.0,0.0,1.0,0.0,1.0,False
3,6,3.0,52.0,14.5,3.6,1.0,518.0,7.0,3.0,1136.0,5.0,0.0,0.0,1.0,0.0,0.0,False
4,2,3.0,52.0,13.8,3.6,1.0,512.0,-1.0,3.0,1136.0,9.0,0.0,0.0,1.0,0.0,0.0,False
5,4,3.0,52.0,7.5,2.3,0.0,479.0,-13.0,3.0,1136.0,9.0,0.0,0.0,1.0,1.0,0.0,False
6,12,5.0,52.5,27.8,4.2,0.0,468.0,-21.0,3.0,1136.0,8.0,0.0,0.0,1.0,1.0,0.0,False
7,1,3.0,56.0,10.9,2.5,1.0,462.0,0.0,3.0,1136.0,13.0,0.0,0.0,1.0,0.0,0.0,False
8,9,5.0,53.5,22.3,3.4,0.0,470.0,-1.0,3.0,1136.0,9.0,0.0,0.0,1.0,0.0,0.0,False
9,5,3.0,52.0,69.5,11.8,0.0,497.0,8.0,3.0,1136.0,16.0,0.0,0.0,1.0,0.0,0.0,False


In [203]:
train.to_csv('clogit_train.csv', encoding='utf-8', index=False)
train_new.to_csv('clogit_train_new.csv', encoding='utf-8', index=False)
test.to_csv('clogit_test.csv', encoding='utf-8', index=False)

In [1]:
import pandas as pd
test = pd.read_csv('clogit_test.csv', encoding='utf-8')

In [11]:
test

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,sex1,sex2
0,2,4,55.0,3.7,1.5,0,466.0,-4.0,4.0,2212,3,0,1,0,0
1,11,4,52.0,35.2,5.7,0,437.0,0.0,4.0,2212,14,0,0,1,0
2,6,3,57.0,2.2,1.2,0,516.0,3.0,4.0,2212,10,0,0,1,0
3,10,4,53.5,77.2,9.2,0,441.0,0.0,4.0,2212,83,0,0,0,0
4,7,3,52.0,6.0,1.8,1,485.0,3.0,4.0,2212,8,0,0,0,0
5,12,3,53.5,23.7,4.5,1,485.0,7.0,4.0,2212,15,0,0,1,0
6,5,4,52.0,46.2,9.4,0,431.0,-8.0,4.0,2212,21,0,0,0,1
7,1,6,53.5,20.1,3.2,0,432.0,0.0,4.0,2212,13,0,0,0,0
8,3,4,51.0,74.0,21.8,0,489.0,-4.0,4.0,2212,10,0,0,0,1
9,4,3,55.0,13.4,3.8,0,467.0,4.0,4.0,2212,11,0,0,0,1


In [10]:
del test['weight_diff'], test['distance'], test['velocity'], test['adj_vel']

In [24]:
test.to_csv('clogit_test.csv', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'clogit_test.csv'

In [26]:
test['phat']=pd.read_csv('dfdsaf.csv', header=None)

In [22]:
f.readline()

'239\n'

In [42]:
test['delete']=test['phat'].apply(lambda x: x == '-')

In [43]:
test_n=test[test['delete'] == 0]

In [30]:
idx = list(set(test_n['groupid']))

In [56]:
dictt=dict(test_n.groupby(['groupid'])['phat'].max())

In [57]:
ids = list(set(test_n['groupid']))

In [66]:
t=0
for i in ids:
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice'])[0] == 1:
        t = t + 1

In [69]:
t/len(ids)

0.32091212458286983

In [51]:
test_n[test_n['groupid']==15]

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,sex1,sex2,phat,delete
17083,2,4,56.5,1.7,1.2,0,454.0,-2.0,2.0,15,17,0,1,0,0,0.375189,False
17084,1,5,56.0,34.2,3.9,0,444.0,9.0,2.0,15,10,0,0,0,0,0.117468,False
17085,11,4,54.0,6.7,1.9,0,479.0,3.0,2.0,15,13,0,0,1,0,0.261923,False
17086,9,4,56.5,11.4,2.4,1,436.0,1.0,2.0,15,31,0,0,0,0,0.137560,False
17087,6,5,52.0,64.0,11.2,0,462.0,10.0,2.0,15,91,0,0,1,0,0.011884,False
17088,3,6,52.5,18.7,4.0,0,461.0,5.0,2.0,15,10,0,0,0,1,0.095975,False


In [48]:
test_n.groupby(['groupid'])['phat'].max()

groupid
8       0.212597
12      0.284630
14      0.266750
15      0.375189
17      0.208919
23      0.313080
26      0.225631
29      0.299129
30      0.209057
31      0.212363
33      0.203817
37      0.188448
45      0.172374
46      0.213429
47      0.226558
49      0.169990
52      0.267901
53      0.159645
63      0.252589
64      0.145870
65      0.283684
67      0.210790
72      0.189327
73      0.211305
74      0.192271
75      0.257784
78      0.169596
80      0.251830
84      0.312298
85      0.222116
          ...   
6179    0.300072
6184    0.254298
6186    0.185301
6187    0.370986
6189    0.225095
6191    0.168692
6193    0.383530
6198    0.226800
6214    0.278623
6218    0.190283
6219    0.231097
6220    0.187663
6224    0.262491
6231    0.197487
6235    0.333239
6236    0.204395
6245    0.250564
6248    0.205917
6250    0.274935
6256    0.294969
6258    0.211082
6259    0.255731
6264    0.274637
6266    0.584682
6273    0.184339
6276    0.226047
6279    0.173482
6284  

In [ ]:
for i in idx:
    dd = test_n[test_n[groupid]==i]
    dd['predicted_win'] =

In [33]:
test_n

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,sex1,sex2,phat,delete


In [46]:
test_n['phat'] = test_n['phat'].apply(float)

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
test_n

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,sex1,sex2,phat,delete
0,2,4,55.0,3.7,1.5,0,466.0,-4.0,4.0,2212,3,0,1,0,0,0.218117729,False
1,11,4,52.0,35.2,5.7,0,437.0,0.0,4.0,2212,14,0,0,1,0,0.049307305,False
2,6,3,57.0,2.2,1.2,0,516.0,3.0,4.0,2212,10,0,0,1,0,0.266419113,False
3,10,4,53.5,77.2,9.2,0,441.0,0.0,4.0,2212,83,0,0,0,0,0.009933067,False
4,7,3,52.0,6.0,1.8,1,485.0,3.0,4.0,2212,8,0,0,0,0,0.141879097,False
5,12,3,53.5,23.7,4.5,1,485.0,7.0,4.0,2212,15,0,0,1,0,0.05988672,False
6,5,4,52.0,46.2,9.4,0,431.0,-8.0,4.0,2212,21,0,0,0,1,0.022399345,False
7,1,6,53.5,20.1,3.2,0,432.0,0.0,4.0,2212,13,0,0,0,0,0.071458854,False
8,3,4,51.0,74.0,21.8,0,489.0,-4.0,4.0,2212,10,0,0,0,1,0.002684421,False
9,4,3,55.0,13.4,3.8,0,467.0,4.0,4.0,2212,11,0,0,0,1,0.12597093,False
